<a href="https://colab.research.google.com/github/kefasmanu/NLP-FELLOWSHIP/blob/main/assignments/Neural_Network_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Model Assignment
The aim of this assignment is to make sure you understand the foundations of model training. We have covered traditional ML and also simple NN. You task is 
1. to write code for training traditional ML model that gives the highest accuracy
2. Code for NN model that gives the highest accuracy

## This we will consider
1. The code works
2. The hyperparameter used for fine tuning - epoch only is not enough
3. The highest accuracy
4. Bonus points for being creative with preprocessing, tokenization and creation of vectors

# Submissions
1. Notebook with code
2. 2 models

# Deadline
Monday 28th at 5pm

## **NEURAL NETWORK**

MOUNTING DRIVE TO COLAB

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
os.chdir("/content/gdrive/MyDrive/NLP FELLOWSHIP/week 6/day 2")
!ls

50k_imdb_movie_reviews.csv  NLPNN.ipynb  saved_weights_linear.pt


Loading the Data

In [16]:
import pandas as pd

full_dataset = pd.read_csv('50k_imdb_movie_reviews.csv')
full_dataset.head()

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test


CHECKING TO SEE IF THE  CLASSES (0,1) IN THE DATA SET IS BALANCED

In [17]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12500
No of negative in train : 12500
No of positive in test : 12500
No of negative in test : 12500


CHECKING FOR DUPLICATES AND IF ANY DROP

In [18]:
full_dataset['review'].describe()


count                                                 50000
unique                                                49582
top       Loved today's show!!! It was a variety and not...
freq                                                      5
Name: review, dtype: object

In [19]:
#Dropping duplicates
full_dataset = full_dataset.drop_duplicates(subset=['review'])
#Results Review after duplicates removal
full_dataset['review'].describe()

count                                                 49582
unique                                                49582
top       I went and saw this movie last night after bei...
freq                                                      1
Name: review, dtype: object

CONFIRMING THE BALANCENESS OF THE TARGET CLASS(0,1) AFTER DROPPING DUPLICATES 

In [20]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12444
No of negative in train : 12337
No of positive in test : 12440
No of negative in test : 12361


TRAIN AND TEST SETS SPLITS

In [21]:
train_dataset = full_dataset[(full_dataset['set'] == 'train')][['review','sentiment']]
test_dataset = full_dataset[(full_dataset['set'] == 'test')][['review','sentiment']]
test_dataset.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


FUNCTION FOR DATA PREPROCESSING

In [23]:
import re
def preprocessing(texts):
  cleaned_text = []
  for text in texts:
    text = text.lower()
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    text = emoji_pattern.sub(r'', text)
    text = url_pattern.sub(r'', text)
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text)
    cleaned_text.append(text)

  return cleaned_text

In [24]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 735.5 MB 10 kB/s 
     |████████████████████████████████| 7.1 MB 23.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.0 which is incompatible.


Dataset transformation to tensors Process

In [25]:
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator


SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # Check this
max_document_length = 300 #hyperparameter

TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Transform Test dataset into tensors

In [26]:
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split()

Transform Train dataset into tensors

In [27]:
torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

Transform text into numbers

In [28]:
max_size = 30000 #hyperparameter
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)

In [29]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 785692), ('the', 326108), ('and', 161663), ('a', 160563), ('of', 144477), ('to', 134325), ('is', 109343), ('it', 92798), ('in', 92099), ('i', 81980), ('this', 72819), ('that', 72538), ("'s", 61640), (' br', 52238), ('was', 49935), ('as', 45797), ('for', 43582), ('with', 43503), ('movie', 42699), ('but', 41298)]


Batch size creation for Both train and test set

In [30]:
BATCH_SIZE = 64 #hyperparameter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True,
    device = device)

Neural network model creation process

In [32]:
import torch.nn as nn
import torch.nn.functional as F


class LR(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2) 
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        
        preds = self.fc3(x) # crossentropyloss handles the softmax
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

Hyper parameters Tuning and Neural Network Model implementation

In [49]:
lr = 1e-4
batch_size = 64
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 42
num_classes = 2

num_epochs = 10
hidden_size = 200
hidden_size1 = 100
hidden_size2 = 50
hidden_size3 = 25

to_train = True

model = LR(max_document_length, hidden_size,hidden_size2, num_classes)

Accuracy Computation function

In [50]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

Model's backpropagation and fine Tuing

In [51]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 34.443 | Train Acc: 50.06%
	 Val. Loss: 11.055 |  Val. Acc: 49.86%
	Train Loss: 4.217 | Train Acc: 51.36%
	 Val. Loss: 1.496 |  Val. Acc: 49.82%
	Train Loss: 1.002 | Train Acc: 50.19%
	 Val. Loss: 0.961 |  Val. Acc: 50.21%
	Train Loss: 0.779 | Train Acc: 50.89%
	 Val. Loss: 0.863 |  Val. Acc: 50.38%
	Train Loss: 0.725 | Train Acc: 50.11%
	 Val. Loss: 0.826 |  Val. Acc: 49.45%
	Train Loss: 0.707 | Train Acc: 50.31%
	 Val. Loss: 0.823 |  Val. Acc: 49.90%
	Train Loss: 0.696 | Train Acc: 50.67%
	 Val. Loss: 0.812 |  Val. Acc: 50.67%
	Train Loss: 0.692 | Train Acc: 51.02%
	 Val. Loss: 0.810 |  Val. Acc: 50.41%
	Train Loss: 0.689 | Train Acc: 50.88%
	 Val. Loss: 0.816 |  Val. Acc: 50.34%
	Train Loss: 0.686 | Train Acc: 51.01%
	 Val. Loss: 0.812 |  Val. Acc: 50.13%


### MODEL 2: Using Embedding Layer as Input to fine tune the result

Neural network function

In [52]:
class MLP(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size2, hidden_size3, hidden_size4, output_dim, dropout, max_document_length):
        super().__init__()
        # embedding and convolution layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(embed_size*max_document_length, hidden_size2)  # dense layer
        self.fc2 = nn.Linear(hidden_size2, hidden_size3)  # dense layer
        self.fc3 = nn.Linear(hidden_size3, hidden_size4)  # dense layer
        self.fc4 = nn.Linear(hidden_size4, output_dim)  # dense layer

    def forward(self, text):
         # text shape = (batch_size, num_sequences)
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
        
        x = embedded.view(embedded.shape[0], -1)  # x = Flatten()(x)
        #embedded = embedded.unsqueeze(1) # fc gets 4 dimension
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        preds = self.fc4(x)
        # preds = F.softmax(preds, 1)
        return preds

Hyperparameters

In [60]:
lr = 1e-5
batch_size = 64
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 42
num_classes = 2

num_epochs = 4
hidden_size = 200
hidden_size1 = 100
hidden_size2 = 50
hidden_size3 = 25

to_train = True

model = MLP(vocab_size, embedding_size, hidden_size1, hidden_size2, hidden_size3,  num_classes, dropout_keep_prob, max_document_length)

Accuracy Computation function

In [61]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

Model Training through Forward and Backward

In [62]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model(text).squeeze(1)
      
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')

	Train Loss: 0.696 | Train Acc: 50.78%
	 Val. Loss: 0.693 |  Val. Acc: 51.37%
	Train Loss: 0.687 | Train Acc: 57.07%
	 Val. Loss: 0.689 |  Val. Acc: 54.55%
	Train Loss: 0.657 | Train Acc: 65.80%
	 Val. Loss: 0.686 |  Val. Acc: 54.86%
	Train Loss: 0.575 | Train Acc: 73.98%
	 Val. Loss: 0.656 |  Val. Acc: 60.95%
